In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_types = ['packet', 'power']
modes = ['baseline', 'music', 'mute', 'questions']

In [10]:
for mode in modes:
    for data_type in data_types:
        path = 'data/' + mode + '/' +  '_'.join([data_type, mode]) + '.csv'
        print(path)

data/baseline/packet_baseline.csv
data/baseline/power_baseline.csv
data/music/packet_music.csv
data/music/power_music.csv
data/mute/packet_mute.csv
data/mute/power_mute.csv
data/questions/packet_questions.csv
data/questions/power_questions.csv


In [11]:
power_baseline_path = 'data/baseline/power_baseline.csv'
power_mute_path = 'data/music/power_music.csv'
power_music_path = 'data/mute/power_mute.csv'
power_questions_path = 'data/questions/power_questions.csv'

In [12]:
power_baseline = pd.read_csv(power_baseline_path)
power_mute = pd.read_csv(power_mute_path)
power_music = pd.read_csv(power_music_path)
power_questions = pd.read_csv(power_questions_path)

In [14]:
power_baseline['state'] = 'baseline'
power_mute['state'] = 'mute'
power_music['state'] = 'music'
power_questions['st']

In [15]:
power_baseline.to_csv(power_baseline_path, index=False)
power_mute.to_csv(power_mute_path, index=False)
power_music.to_csv(power_music_path, index=False)
power_questions.to_csv(power_questions_path, index=False)

,apower,voltage,current,time,state
0,1.4,120.3,0.029,1669868506,baseline
1,1.4,120.2,0.029,1669868507,baseline
2,1.4,120.2,0.029,1669868508,baseline
3,1.4,120.2,0.028,1669868509,baseline
4,1.4,120.2,0.028,1669868510,baseline
...,...,...,...,...,...
105,1.5,120.3,0.029,1669868619,baseline
106,1.5,120.3,0.029,1669868620,baseline
107,1.4,120.3,0.029,1669868621,baseline
108,1.4,120.3,0.029,1669868622,baseline
